In [1]:
// run this cell to prevent Jupyter from displaying the null output cell
com.twosigma.beakerx.kernel.Kernel.showNullExecutionResult = false;

<a id="notebook_id"></a>
# Substitutability

Inheritance models the is-a or is-substitutable-for relationship. What this means is that a subtype object should always be usable whenever a supertype object is required, and doing so will not cause a program to behave unexpectedly.

In object-oriented programming, the term *substitutability* often refers to the Liskov substitution principle (named after [Barbara Liskov](https://en.wikipedia.org/wiki/Barbara_Liskov) who first described the principle). The formal definition of the principle is

> Let $\phi(x)$ be a property provable about objects $x$ of type T. Then $\phi(y)$ should be true for objects $y$ of type S where S is a subtype of T. 

In practice, Liskov substitutability means that subclass objects must behave the same as objects of their supertype. Object behaviour is determined by the object's methods; thus ensuring substitutability requires paying careful attention when overriding methods in a subclass.

## Method parameters

Many methods place restrictions on their parameter values. 

An overridden method of a subclass needs to accept the same input parameter values as the method of the superclass. That means you can implement less restrictive validation rules, but you are not allowed to enforce stricter ones in your subclass. Otherwise, any code that calls this method on an object of the superclass might cause an exception, if it gets called with an object of the subclass.


## Method return values 

Similar rules apply to the return value of the method. The return value of a method of the subclass needs to comply with the same rules as the return value of the method of the superclass. You can only decide to apply even stricter rules by returning a specific subclass of the defined return value, or by returning a subset of the valid return values of the superclass.


## Class invariants